In [1]:
import boto3

sagemaker = boto3.client('sagemaker')

model_name='spark-regression-model-001'
endpoint_config_name='inference-server-001a-config'
endpoint_name='inference-server-001a'

## Create SageMaker Model
# Update Role Arn
role = 'arn:aws:iam::036983998592:role/service-role/AmazonSageMaker-ExecutionRole-20180518T114698'
primary_container = {
# Do not change the inference image link unless you have your own
    'Image': '959874710265.dkr.ecr.us-west-2.amazonaws.com/inference-server:latest',
#    'ModelDataUrl': 's3://ruchika-wibd/models2/spark-regression-model/model.tgz'
    'ModelDataUrl': 's3://ruchika-wibd-west-2/mleap/models/pipeline1/model.tgz'
}

create_model_response = sagemaker.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-west-2:036983998592:model/spark-regression-model-001


In [2]:
## Create EndPoint Config
response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'default-variant-name',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        },
    ])

print (response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:036983998592:endpoint-config/inference-server-001a-config', 'ResponseMetadata': {'RequestId': 'ccb244c7-1c41-43a4-b1b7-dcc3437ad161', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ccb244c7-1c41-43a4-b1b7-dcc3437ad161', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Tue, 28 Aug 2018 15:04:51 GMT'}, 'RetryAttempts': 0}}


In [3]:
## Create Sagemaker Endpoint
## Wait for Response. This may take a few minutes. 
response = sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print (response)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:036983998592:endpoint/inference-server-001a', 'ResponseMetadata': {'RequestId': '5598faf2-c3da-46ae-9310-0f2297439785', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5598faf2-c3da-46ae-9310-0f2297439785', 'content-type': 'application/x-amz-json-1.1', 'content-length': '89', 'date': 'Tue, 28 Aug 2018 15:04:57 GMT'}, 'RetryAttempts': 0}}


In [4]:
%%time
## Test the Endpoint
import boto3, io, json

client = boto3.client('sagemaker-runtime')

input='{"schema":{"fields":[{"name":"Price","type":"double"},{"name":"Mileage","type":"integer"}, \
{"name":"Make","type":"string"},{"name":"Model","type":"string"},{"name":"Trim","type":"string"}, \
{"name":"Type","type":"string"},{"name":"Cylinder","type":"integer"},{"name":"Liter","type":"double"}, \
{"name":"Doors","type":"integer"},{"name":"Cruise","type":"integer"},{"name":"Sound","type":"integer"}, \
{"name":"Leather","type":"integer"}]}, \
"rows":[[9041.9062544231,26191,"Chevrolet","AVEO","SVM Sedan 4D","Sedan",4,1.6,4,0,0,1]]}'
    
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=input,
    ContentType='application/json',
    Accept='application/json'
)

res_json = json.loads(response['Body'].read().decode("utf-8"))

print (json.dumps(res_json, indent=2))

{
  "schema": {
    "fields": [
      {
        "name": "prediction",
        "type": {
          "type": "basic",
          "base": "double",
          "isNullable": false
        }
      }
    ]
  },
  "rows": [
    [
      10236.175823272792
    ]
  ]
}
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 2.52 s
